In [1]:
getwd()
setwd(paste0(getwd(), '/..'))
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/NA_fix"

[1] "/mnt/hdd/develop/kaggle/house-prices/R"

In [32]:
source('helpers.R')

kaggle.house.loadLibraries()
data <- kaggle.house.loadData()

df.LotFrontage <- data$combined %>% filter(!is.na(LotFrontage))

df.LotFrontage.vs.numeric <- df.LotFrontage %>% select_if(is.numeric)

df.LotFrontage.vs.char <- df.LotFrontage %>% 
    select(starts_with('LotFrontage'),  
           which(sapply(df.LotFrontage, is.character)) %>% names)

df.LotFrontage %>% dim
df.LotFrontage.vs.numeric %>% dim
df.LotFrontage.vs.char %>% dim

[1] 2433   80

[1] 2433   35

[1] 2433   46

In [46]:
df.LotFrontage.vs.numeric %>%
gather(attr.name, attr.val, -LotFrontage) %>%
filter(!is.na(attr.val)) %>%
mutate(
    attr.val.log = ifelse(attr.val > 0, log(attr.val), 0),
    LotFrontage.log = log(LotFrontage)
) -> df.LotFrontage.vs.numeric.long

df.LotFrontage.vs.numeric.long %>% head
df.LotFrontage.vs.numeric.long %>% count

LotFrontage,attr.name,attr.val,attr.val.log,LotFrontage.log
65,LotArea,8450,9.041922,4.174387
80,LotArea,9600,9.169518,4.382027
68,LotArea,11250,9.328123,4.219508
60,LotArea,9550,9.164296,4.094345
84,LotArea,14260,9.565214,4.430817
85,LotArea,14115,9.554993,4.442651


n
82547


In [47]:
df.LotFrontage.vs.numeric.long %>% 
group_by(attr.name) %>% 
nest %>% 
mutate(
    mod = map(data, ~lm(LotFrontage ~ attr.val, .)), 
    glance = map(mod, broom::glance),
    rsq = glance %>% map_dbl("r.squared"),
    
    mod2 = map(data, ~lm(LotFrontage ~ attr.val.log, .)), 
    glance2 = map(mod2, broom::glance),
    rsq2 = glance2 %>% map_dbl("r.squared"),
    
    mod3 = map(data, ~lm(LotFrontage.log ~ attr.val.log, .)), 
    glance3 = map(mod3, broom::glance),
    rsq3 = glance3 %>% map_dbl("r.squared")
) %>%
arrange(desc(rsq)) %>%
select(attr.name, rsq, rsq2, rsq3) %>%
print

# A tibble: 34 x 4
      attr.name        rsq        rsq2        rsq3
          <chr>      <dbl>       <dbl>       <dbl>
 1      LotArea 0.23999767 0.467189030 0.587193938
 2    X1stFlrSF 0.20999074 0.210587817 0.218847392
 3    GrLivArea 0.14627714 0.125415343 0.108557166
 4   GarageArea 0.12944610 0.033312216 0.033526172
 5  TotalBsmtSF 0.12589894 0.008013407 0.007024319
 6 TotRmsAbvGrd 0.12215963 0.123642972 0.126792650
 7   GarageCars 0.09646429 0.090350458 0.077319103
 8   Fireplaces 0.06862818 0.030197358 0.022495100
 9 BedroomAbvGr 0.05517420 0.060886185 0.074960584
10   MasVnrArea 0.04887582 0.038520767 0.023899711
# ... with 24 more rows


In [48]:
df.LotFrontage.vs.numeric %>%
gather(attr.name, attr.val, -LotFrontage, -LotArea) %>%
filter(!is.na(attr.val)) %>%
mutate(
    attr.val.log = ifelse(attr.val > 0, log(attr.val), 0),
    LotFrontage.log = log(LotFrontage),
    LotArea.log = log(LotArea)
) %>%
select(-LotFrontage, -LotArea) -> df.LotFrontage.vs.numeric.long2

df.LotFrontage.vs.numeric.long2 %>% head
df.LotFrontage.vs.numeric.long2 %>% count

attr.name,attr.val,attr.val.log,LotFrontage.log,LotArea.log
OverallQual,7,1.945910,4.174387,9.041922
OverallQual,6,1.791759,4.382027,9.169518
OverallQual,7,1.945910,4.219508,9.328123
OverallQual,7,1.945910,4.094345,9.164296
OverallQual,8,2.079442,4.430817,9.565214
OverallQual,5,1.609438,4.442651,9.554993


n
80114


In [49]:
df.LotFrontage.vs.numeric.long2 %>% 
group_by(attr.name) %>% 
nest %>% 
mutate(
    mod = map(data, ~lm(LotFrontage.log ~ LotArea.log, .)), 
    glance = map(mod, broom::glance),
    rsq = glance %>% map_dbl("r.squared"),
    
    mod1 = map(data, ~lm(LotFrontage.log ~ LotArea.log + attr.val, .)), 
    glance1 = map(mod1, broom::glance),
    rsq1 = glance1 %>% map_dbl("r.squared"),
    
    mod2 = map(data, ~lm(LotFrontage.log ~ LotArea.log + attr.val.log, .)), 
    glance2 = map(mod2, broom::glance),
    rsq2 = glance2 %>% map_dbl("r.squared")
) %>%
arrange(desc(rsq2)) %>%
select(attr.name, rsq, rsq1, rsq2) %>%
print

# A tibble: 33 x 4
      attr.name       rsq      rsq1      rsq2
          <chr>     <dbl>     <dbl>     <dbl>
 1    X1stFlrSF 0.5871939 0.5952974 0.5986197
 2   MasVnrArea 0.5886948 0.5922331 0.5943946
 3 BedroomAbvGr 0.5871939 0.5912319 0.5936016
 4 TotRmsAbvGrd 0.5871939 0.5918779 0.5929072
 5   GarageCars 0.5873755 0.5915473 0.5908690
 6    X2ndFlrSF 0.5871939 0.5878356 0.5897907
 7    BsmtUnfSF 0.5882109 0.5912117 0.5895906
 8    GrLivArea 0.5871939 0.5891953 0.5892236
 9  TotalBsmtSF 0.5882109 0.5926353 0.5885418
10  OverallQual 0.5871939 0.5888413 0.5885256
# ... with 23 more rows


In [59]:
df.LotFrontage.vs.char %>%
gather(attr.name, attr.val, -LotFrontage) %>%
filter(!is.na(attr.val)) %>%
group_by(attr.name) %>%
filter(n_distinct(attr.val) > 1) %>%
mutate(
    LotFrontage.log = log(LotFrontage)
) -> df.LotFrontage.vs.char.long

df.LotFrontage.vs.char.long %>% head
df.LotFrontage.vs.char.long %>% dim



LotFrontage,attr.name,attr.val,LotFrontage.log
65,MSSubClass,60,4.174387
80,MSSubClass,20,4.382027
68,MSSubClass,60,4.219508
60,MSSubClass,70,4.094345
84,MSSubClass,60,4.430817
85,MSSubClass,50,4.442651


[1] 95865     4

In [61]:
df.LotFrontage.vs.char.long %>% 
group_by(attr.name) %>% 
nest %>% 
mutate(
    mod1 = map(data, ~lm(LotFrontage ~ attr.val, .)), 
    glance1 = map(mod1, broom::glance),
    rsq1 = glance1 %>% map_dbl("r.squared"),
    
    mod2 = map(data, ~lm(LotFrontage.log ~ attr.val, .)), 
    glance2 = map(mod2, broom::glance),
    rsq2 = glance2 %>% map_dbl("r.squared")
) %>%
arrange(desc(rsq2)) %>%
select(attr.name, rsq1, rsq2) %>%
print

# A tibble: 44 x 3
      attr.name       rsq1       rsq2
          <chr>      <dbl>      <dbl>
 1   MSSubClass 0.35397282 0.56689967
 2     BldgType 0.27873183 0.49821725
 3 Neighborhood 0.26761868 0.41942559
 4        Alley 0.15870576 0.22724337
 5     MSZoning 0.13744685 0.18531266
 6       PoolQC 0.17120892 0.17614046
 7  Exterior2nd 0.06735684 0.10855302
 8   GarageType 0.09624769 0.08976496
 9    LotConfig 0.07566657 0.07174850
10  Exterior1st 0.04085280 0.05812463
# ... with 34 more rows


In [89]:
df.LotFrontage %>%
select(LotFrontage, LotArea, MSSubClass, BldgType, Neighborhood) %>%
na.omit %>%
mutate(
    LotArea.log = log(LotArea),
    LotFrontage.log = log(LotFrontage)
) -> df

df.LotFrontage %>% dim
df %>% dim

lm(LotFrontage.log ~ LotArea.log, data = df) %>% summary %>% `$`("r.squared")

lm(LotFrontage.log ~ LotArea.log + MSSubClass, data = df) %>% summary %>% `$`("r.squared")
lm(LotFrontage.log ~ LotArea.log + MSSubClass + BldgType, data = df) %>% summary %>% `$`("r.squared")
lm(LotFrontage.log ~ LotArea.log + MSSubClass + Neighborhood, data = df) %>% summary %>% `$`("r.squared")
lm(LotFrontage.log ~ LotArea.log + MSSubClass + BldgType + Neighborhood, data = df) %>% summary %>% `$`("r.squared")
print('---')
lm(LotFrontage.log ~ LotArea.log + BldgType, data = df) %>% summary %>% `$`("r.squared")
lm(LotFrontage.log ~ LotArea.log + BldgType, data = df) %>% summary %>% `$`("r.squared")
lm(LotFrontage.log ~ LotArea.log + BldgType + Neighborhood, data = df) %>% summary %>% `$`("r.squared")
print('---')
lm(LotFrontage.log ~ LotArea.log + Neighborhood, data = df) %>% summary %>% `$`("r.squared")

[1] 2433   80

[1] 2433    7

[1] 0.5871939

[1] 0.6668896

[1] 0.6724534

...

[1] 0.646077

[1] 0.646077

[1] 0.6969419

[1] "---"


[1] 0.6349365